In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import dill
import time
import numpy as np
import pandas as pd
import scipy.io as io
import datetime
from matplotlib import pyplot as plt

sys.path.append('../helper_functions')
sys.path.append('helper_functions')
sys.path.append('../objects')
sys.path.append('objects')

In [3]:
from FishGeneration import FishGeneration
from Fish_IceCreamCone import IceCreamConeFish
from Fish_Capsule import CapsuleFish

from assertion_tests import run_tests
run_tests()

'Success!'

## Load fish details

#### the EOD waveform

In [4]:
# use a dummy EOD until real EOD is available in this framework
# full_eod = np.linspace(0, 2 * np.pi, 1000)
# full_eod = np.sin(full_eod) * full_eod * np.exp(-((full_eod - np.pi) ** 2) / (2 * (np.pi / 6) ** 2))
full_eod = np.genfromtxt("../../efish-receptors-model/data/eod-waveform/eod-waveform-fitted.csv", delimiter=",")
full_eod.shape

(1005,)

#### the receptor filters

In [5]:
# use dummy filters until real filters are available in this framework
receptor_filters = pd.read_csv("../../efish-receptors-model/data/filters/fish-filters-0p4.csv").T.to_numpy()
receptor_filters.shape

(2, 1005)

#### check the input arguments for the wanted fish object (helps with creating fish objects)

In [6]:
for x in CapsuleFish._initialize_input_argument_names():
    print(f'{x},')

conductivity=1,
relative_permittivity=1,
receptors_locations=np.array([]).reshape(0,3),
receptors_normals=np.array([]).reshape(0,3),
point_currents_magnitudes=np.array([]).reshape(0),
point_currents_locations=np.array([]).reshape(0,3),
receptor_filters=np.array([]).reshape(0,1000),
eod_wave_form=np.arange(1000),
skin_resistivity=1,
sampling_rate=(2.5, "M"),  # MHz,
eod_delay=(0,), # s,
nose_position=[0,0,0],
fish_length=1,
angle_yaw=0,
angle_pitch=0,
angle_roll=0,
relative_bend_locations_percentage=np.array([]),
relative_bend_angle_lateral=(np.array([]),"deg"),
relative_bend_angle_dorso_ventral =(np.array([]),"deg"),
point_currents_range_percentage=dict(start=0, end=100),
N_point_currents=101,
point_currents_magnitude_scale=1,
receptors_init=dict(method="random", head=100, body=200),
vertical_semi_axis,
lateral_semi_axis=None,
rostrocaudal_semi_axis=None,
rostrocaudal_semi_axis_tail=None,


## Dataset: YYY_MM_DD Purpose

## Dataset: 2024_06_05 Characterization dataset

In [7]:
dataset_name = 'data-2024_06_05-characterization_dataset'

In [12]:
base_fish = CapsuleFish(
    receptor_filters=receptor_filters,
    eod_wave_form=full_eod,
    skin_resistivity=(3, "k", "c.2"),  # 3 kOhm.cm^2
    sampling_rate=(2.5, "M"),  # 2.5 MHz,
    nose_position=[0, 0, 0],
    fish_length=(15, "c"),
    angle_yaw=(0, "deg"),
    angle_pitch=(0, "deg"),
    angle_roll=(0, "deg"),
    relative_bend_locations_percentage=np.array([]),
    relative_bend_angle_lateral=(np.array([]), "deg"),
    relative_bend_angle_dorso_ventral=(np.array([]), "deg"),
    receptors_init=dict(
        method="grid_uniDense",
        head_t=0,
        head_u=0,
        tail_t=0,
        tail_u=0,
        body_t=20,
        body_u=30,
    ),
    vertical_semi_axis=(2.0, "c"),
    lateral_semi_axis=(0.9, "c"),
    rostrocaudal_semi_axis=0,
    rostrocaudal_semi_axis_tail=(0.9, "c"),
)

dill.dump(base_fish, open(f"base-fish/{dataset_name}.pkl", "wb"), protocol=4)
base_fish = dill.load(open(f"base-fish/{dataset_name}.pkl", "rb"))

Success!


In [13]:
base_fish.visualize_scatter(show_normals=2, show_point_currents=3, fig_width=1000, fig_height=350)
print(base_fish.get_N_receptors())

630


In [66]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict["save_name"]                       = dataset_name
# aquarium properties
data_params_dict["water_conductivities"]            = [0.01]  # Siemens / meter
data_params_dict["boundary_displacements"]          = [[]]
data_params_dict["boundary_normals"]                = [[]]
# fish properties
data_params_dict["fish_obj"]                        = base_fish
data_params_dict["fish_bend_location_percentages"]  = [[]]
data_params_dict["fish_bend_angle_lateral_deg"]         = [[]]
data_params_dict["fish_bend_angle_dorso_ventral_deg"]   = [[]]
data_params_dict["fish_yaw_deg"]                        = [0]
data_params_dict["fish_pitch_deg"]                      = [0]
data_params_dict["fish_roll_deg"]                       = [0]
# worm properties
data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(2), np.log(100), 40)) * 1e3
data_params_dict["worm_capacitances"]               = np.exp(np.linspace(np.log(2), np.log(100), 40)) * 1e-10
data_params_dict["worm_radii"]                      = np.arange(1, 8.5, 0.5) * 1e-3
data_params_dict["worm_position_xs"]                = np.arange(5, -95, -5) * 1e-3
data_params_dict["worm_position_ys"]                = np.arange(8, 32) * 1e-3 + base_fish.get_lateral_semi_axis()
data_params_dict["worm_position_zs"]                = np.linspace(-20, 20, 9) * 1e-3
# save params file
dill.dump(data_params_dict, open(f"params/{data_params_dict['save_name']}.pkl", "wb"), protocol=4)

## Dataset: 2024_06_05 R&C feature space

In [7]:
# print(datetime.datetime.now().strftime('%Y_%m_%d'))
# 2024_06_05

In [8]:
dataset_name = 'data-2024_06_05-R_and_C_feature_space'

### create the Fish object

In [9]:
base_fish = CapsuleFish(
    receptor_filters=receptor_filters,
    eod_wave_form=full_eod,
    skin_resistivity=(3, "k", "c.2"),  # 3 kOhm.cm^2
    sampling_rate=(2.5, "M"),  # 2.5 MHz,
    nose_position=[0, 0, 0],
    fish_length=(15, "c"),
    angle_yaw=(0, "deg"),
    angle_pitch=(0, "deg"),
    angle_roll=(0, "deg"),
    relative_bend_locations_percentage=np.array([]),
    relative_bend_angle_lateral=(np.array([]), "deg"),
    relative_bend_angle_dorso_ventral=(np.array([]), "deg"),
    receptors_init=dict(
        method="manual",
        locations=np.array([[15e-2 - 18e-3, 9e-3, 0], [15e-2 - 20e-3, 9e-3, 0], [15e-2 - 22e-3, 9e-3, 0]]).reshape(
            -1, 3
        ),
        normals=np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0]]).reshape(-1, 3),
    ),
)

dill.dump(base_fish, open(f"base-fish/{dataset_name}.pkl", "wb"), protocol=4)
base_fish = dill.load(open(f"base-fish/{dataset_name}.pkl", "rb"))

Success!


In [10]:
base_fish.visualize_scatter(show_normals=2, show_point_currents=3, fig_width=1000, fig_height=350)
print(base_fish.get_N_receptors())

3


In [11]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict["save_name"]                       = dataset_name
# aquarium properties
data_params_dict["water_conductivities"]            = [0.005, 0.01, 0.02, 0.04, 0.06]  # Siemens / meter
data_params_dict["boundary_displacements"]          = [[]]
data_params_dict["boundary_normals"]                = [[]]
# fish properties
data_params_dict["fish_obj"]                        = base_fish
data_params_dict["fish_bend_location_percentages"]  = [[]]
data_params_dict["fish_bend_angle_lateral_deg"]         = [[]]
data_params_dict["fish_bend_angle_dorso_ventral_deg"]   = [[]]
data_params_dict["fish_yaw_deg"]                        = [0]
data_params_dict["fish_pitch_deg"]                      = [0]
data_params_dict["fish_roll_deg"]                       = [0]
# worm properties
data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(2), np.log(100), 100)) * 1e3
data_params_dict["worm_capacitances"]               = np.exp(np.linspace(np.log(2), np.log(100), 100)) * 1e-10
data_params_dict["worm_radii"]                      = np.array([5]) * 1e-3
data_params_dict["worm_position_xs"]                = np.array([-20]) * 1e-3
data_params_dict["worm_position_ys"]                = (5 * np.power(np.sqrt(2), np.arange(9))).round(0) * 1e-3 + base_fish.get_lateral_semi_axis() + data_params_dict["worm_radii"]
data_params_dict["worm_position_zs"]                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open(f"params/{data_params_dict['save_name']}.pkl", "wb"), protocol=4)

## Dataset: 2024_05_23 Test

In [ ]:
print(datetime.datetime.now().strftime('%Y_%m_%d'))

In [ ]:
dataset_name = 'data-2024_05_23-test'

#### create the Fish object

In [ ]:
base_fish = CapsuleFish(
    receptor_filters=receptor_filters,
    eod_wave_form=full_eod,
    skin_resistivity=(3, "k", "c.2"),  # 3 kOhm.cm^2
    sampling_rate=(2.5, "M"),  # 2.5 MHz,
    nose_position=[0, 0, 0],
    fish_length=(15, "c"),
    angle_yaw=(0, "deg"),
    angle_pitch=(0, "deg"),
    angle_roll=(0, "deg"),
    relative_bend_locations_percentage=np.array([]),
    relative_bend_angle_lateral=(np.array([]), "deg"),
    relative_bend_angle_dorso_ventral=(np.array([]), "deg"),
    # receptors_init=dict(
    #     method="grid_uniDense",
    #     head_t=0,
    #     head_u=0,
    #     tail_t=0,
    #     tail_u=0,
    #     body_t=60,
    #     body_u=30,
    # ),
    # vertical_semi_axis=(2.0, "c"),
    # lateral_semi_axis=(0.9, "c"),
    # rostrocaudal_semi_axis=0,
    # rostrocaudal_semi_axis_tail=0,
    receptors_init=dict(
        method="manual",
        locations=np.array([[15e-2 - 18e-3, 9e-3, 0], [15e-2 - 20e-3, 9e-3, 0], [15e-2 - 22e-3, 9e-3, 0]]).reshape(
            -1, 3
        ),
        normals=np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0]]).reshape(-1, 3),
    ),
)

dill.dump(base_fish, open(f"base-fish/{dataset_name}.pkl", "wb"), protocol=4)
base_fish = dill.load(open(f"base-fish/{dataset_name}.pkl", "rb"))

#### visualize fish object for inspection

In [ ]:
base_fish.visualize_scatter(show_normals=2, show_point_currents=3, fig_width=1000, fig_height=350)
print(base_fish.get_N_receptors())

#### select simulation parameters

In [ ]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict["save_name"]                       = dataset_name
# aquarium properties
data_params_dict["water_conductivities"]            = [0.01, 0.015]
data_params_dict["boundary_displacements"]          = [[-1e9], [-30e-3, -50e-3]]
data_params_dict["boundary_normals"]                = [[0, 0, 1], [0, 0, 1]]
# fish properties
data_params_dict["fish_obj"]                        = base_fish
data_params_dict["fish_bend_location_percentages"]  = [[50], []]
data_params_dict["fish_bend_angle_lateral_deg"]         = [[20], [45], []]
data_params_dict["fish_bend_angle_dorso_ventral_deg"]   = [[20], [30], []]
data_params_dict["fish_yaw_deg"]                        = [0]
data_params_dict["fish_pitch_deg"]                      = [0]
data_params_dict["fish_roll_deg"]                       = [0]
# worm properties
data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(2), np.log(100), 10)) * 1e3
data_params_dict["worm_capacitances"]               = np.exp(np.linspace(np.log(2), np.log(100), 10)) * 1e-10
data_params_dict["worm_radii"]                      = np.array([5, 12.5]) * 1e-3
data_params_dict["worm_position_xs"]                = np.array([-30]) * 1e-3
data_params_dict["worm_position_ys"]                = np.array([5, 10]) * 1e-3 + base_fish.get_lateral_semi_axis() + data_params_dict["worm_radii"]
data_params_dict["worm_position_zs"]                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open(f"params/{data_params_dict['save_name']}.pkl", "wb"), protocol=4)

## Misc - previous param dicts for reference

In [ ]:
# Create Data Params Dict
data_params_dict = {}
# data_params_dict['save_file_name']                  = 'data_240411_SpatialProcessing_MovingObject_7by15'
data_params_dict['save_file_name']                  = 'data_240425_Detection_1050_receptors_HighContrast'
# aquarium properties
data_params_dict['water_conductivities']            = [0.01]
data_params_dict['boundary_displacements']          = [[-1e9]]
data_params_dict['boundary_normals']                =  [[0,0,1]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
# data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e3
data_params_dict['worm_resistances']                = np.array([2e3, 100e3])  # HighContrast
data_params_dict['worm_capacitances']               = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e-10  # HighContrast when paired with HighContrast resistances
# data_params_dict['worm_capacitances']               = np.array([2e-10])  # HighContrast
data_params_dict['worm_radii']                      = np.array([12.5]) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-30]) * 1e-3   # - np.arange(101) * 3e-4
data_params_dict['worm_position_ys']                = (5 * np.power(np.sqrt(1.33), np.arange(25))).round(0) * 1e-3 + base_fish.lat_ax + data_params_dict['worm_radii']
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

In [ ]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict['save_file_name']                  = 'data_231116_SpatialProcessing_MovingObject_11by25'
# aquarium properties
data_params_dict['water_conductivities']            = [0.01]  # np.array([100, 300, 600]) * 1e-6 / 1e-2
data_params_dict['boundary_displacements']          = [[-1e9]]
                                                      # [[-1e9]+list(-np.arange(35, 70, 5)[::-1]*1e-3),
                                                      #  [-1e9]+list(-np.arange(35, 70, 5)[::-1]*1e-3), 
                                                      #  [-1e9]+list(-np.arange(35, 70, 5)[::-1]*1e-3)]
                                                    #  [[-1e9]+list(-np.linspace(5*base_fish.roc_ax, base_fish.roc_ax, 9)), ]  # 
                                                    #    [-1e9]+list(-np.linspace(0.058, 0.022, 9)),
                                                    #    [-1e9]+list(-np.linspace(0.058, 0.022, 9))]
data_params_dict['boundary_normals']                =  [[0,0,1]]
                                                      # [[0,0,1], [0,-1,0], [0,1,0]]
      #  [[0,0,1], ]  #
      #    [0,-1,0],
      #    [0,1,0]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]#[[x] for x in np.linspace(-60,60,25)]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]#[[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]#[[60]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e3
data_params_dict['worm_capacitances']               = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e-10
data_params_dict['worm_radii']                      = np.array([12.5]) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-30]) * 1e-3 - np.arange(11) * 3e-3
data_params_dict['worm_position_ys']                = np.array([5,10,15,30,50,70,100,150]) * 1e-3 + base_fish.lat_ax + data_params_dict['worm_radii']
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

In [ ]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict['save_file_name']                  = 'data_240311_falloff_with_distance_and_radius'
# aquarium properties
data_params_dict['water_conductivities']            = np.array([100]) * 1e-6 / 1e-2
data_params_dict['boundary_displacements']          = [[-1e9]]
data_params_dict['boundary_normals']                =  [[0,0,1]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
data_params_dict['worm_resistances']                = np.array([500,50000])
data_params_dict['worm_capacitances']               = np.array([2e-10])
data_params_dict['worm_radii']                      = np.arange(5,11) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-30]) * 1e-3
data_params_dict['worm_position_ys']                = np.arange(10,201) * 1e-3 + base_fish.lat_ax
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)